# Chatbot Projesi
Bu proje, Python programlama dili kullanarak bir chatbot oluşturmayı amaçlamaktadır. Chatbot, kullanıcılarla insan benzeri etkileşimler kurarak, işletmelerin günlük olarak aldıkları en yaygın soruları çözmelerine yardımcı olur. Bu çalışma, başlangıç seviyesindeki geliştiriciler için tasarlanmıştır ve NLTK kütüphanesi kullanılarak bir chatbot'un nasıl oluşturulacağını adım adım öğreneceksiniz.

In [2]:
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
import json
import random

In [3]:
{
  "intents": [
    {
      "tag": "greeting",
      "patterns": ["Hello", "Hi", "How are you?", "Good day", "Good morning", "Good evening"],
      "responses": ["Hello! How can I assist you?", "Hi! How can I help you today?", "Good day! What can I do for you?"]
    },
    {
      "tag": "goodbye",
      "patterns": ["Goodbye", "Bye", "See you", "See you later", "Logout"],
      "responses": ["Goodbye! Have a nice day!", "See you soon!", "Feel free to ask if you need more help!"]
    },
    {
      "tag": "thanks",
      "patterns": ["Thank you", "Thanks", "You helped a lot", "You're awesome", "You're amazing"],
      "responses": ["You're welcome!", "My pleasure!", "Happy to help!"]
    },
    {
      "tag": "weather",
      "patterns": ["How's the weather?", "Will it rain today?", "What's the temperature?", "What's the wind speed?"],
      "responses": ["Currently sunny and 25°C.", "No rain expected today.", "Temperature ranges between 22°C to 28°C."]
    },
    {
      "tag": "appointment",
      "patterns": ["I need an appointment", "Book a doctor", "Any available time?", "Can I get an appointment today?"],
      "responses": ["Please visit our website for appointments.", "Next available slot is tomorrow at 2 PM."]
    },
    {
      "tag": "product_info",
      "patterns": ["What's the price?", "Any discounts?", "Is it in stock?", "Can I see product specs?"],
      "responses": ["This product costs $499.", "Currently 20% off!", "Available in stock."]
    },
    {
      "tag": "complaint",
      "patterns": ["I have a complaint", "Defective product", "Not satisfied", "How to return?"],
      "responses": ["We're here to resolve your issue.", "Please email support@company.com."]
    },
    {
      "tag": "humor",
      "patterns": ["Tell a joke", "Say something funny", "Make me laugh", "Can you be funny?"],
      "responses": ["Why did the chicken cross the road? To get to the other side!", "As an AI, I try to be funny. Did it work?"]
    },
    {
      "tag": "time",
      "patterns": ["What time is it?", "Current time?", "What's the time?"],
      "responses": ["It's 3:30 PM now.", "The time is 3:30."]
    },
    {
      "tag": "help",
      "patterns": ["I need help", "How to use?", "Need support", "Can you guide me?"],
      "responses": ["How can I assist you?", "Please explain your issue in detail."]
    }
  ]
}

{'intents': [{'tag': 'greeting',
   'patterns': ['Hello',
    'Hi',
    'How are you?',
    'Good day',
    'Good morning',
    'Good evening'],
   'responses': ['Hello! How can I assist you?',
    'Hi! How can I help you today?',
    'Good day! What can I do for you?']},
  {'tag': 'goodbye',
   'patterns': ['Goodbye', 'Bye', 'See you', 'See you later', 'Logout'],
   'responses': ['Goodbye! Have a nice day!',
    'See you soon!',
    'Feel free to ask if you need more help!']},
  {'tag': 'thanks',
   'patterns': ['Thank you',
    'Thanks',
    'You helped a lot',
    "You're awesome",
    "You're amazing"],
   'responses': ["You're welcome!", 'My pleasure!', 'Happy to help!']},
  {'tag': 'weather',
   'patterns': ["How's the weather?",
    'Will it rain today?',
    "What's the temperature?",
    "What's the wind speed?"],
   'responses': ['Currently sunny and 25°C.',
    'No rain expected today.',
    'Temperature ranges between 22°C to 28°C.']},
  {'tag': 'appointment',
   'patterns'

In [4]:
lemmatizer = WordNetLemmatizer()
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BETÜL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\BETÜL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\BETÜL\AppData\Roaming\nltk_data...


True

In [5]:
# JSON dosyasını yükle
with open('intents.json.txt.txt', encoding='utf-8') as file:
    intents = json.load(file)

words = []
classes = []
documents = []

# Cümleleri tokenize et ve köklerine ayır
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        documents.append((word, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ["?", "!"]]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [6]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [7]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))  # Düzeltildi
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200


C:\Users\BETÜL\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0762 - loss: 2.4158      
Epoch 2/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0712 - loss: 2.2753     
Epoch 3/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0914 - loss: 2.3435     
Epoch 4/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1182 - loss: 2.2585     
Epoch 5/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1213 - loss: 2.2337     
Epoch 6/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2545 - loss: 2.1754 
Epoch 7/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3413 - loss: 2.0668 
Epoch 8/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2631 - loss: 2.1328 
Epoch 9/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3679 - loss: 2.0570 
Epoch 10/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3483 - loss: 2.0050 
Epoch 11/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4534 - loss: 1.8438 
Epoch 12/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4

In [8]:
def clean_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    results = [[i, r] for i, r in enumerate(res) if r > 0.25]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

# Sohbeti başlat
print("Chatbot aktif! (Çıkmak için 'quit' yazın)")
while True:
    user_input = input("Sen: ")
    if user_input.lower() == "quit":
        break
    intents_list = predict_class(user_input)
    response = get_response(intents_list, intents)
    print("Bot:", response)

Chatbot aktif! (Çıkmak için 'quit' yazın)


Sen:  hello


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Bot: Hello! How can I assist you?


Sen:  Tell a joke


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Bot: Why did the chicken cross the road? To get to the other side!


Sen:  quit


In [9]:
import pickle

with open('chatbot_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential


# Save the model in H5 format
model.save('chatbot_model.h5')
print("Model successfully converted and saved as chatbot_model.h5")

Model successfully converted and saved as chatbot_model.h5


In [11]:
with open('words_classes.pkl', 'wb') as f:
    pickle.dump((words, classes), f)